In [1]:
import pandas as pd
import numpy as np
from PIL import Image
import cv2
import json
import os

In [2]:
def getFrames(videoPath):
    cap = cv2.VideoCapture(videoPath)
    cap.set(cv2.CAP_PROP_POS_AVI_RATIO,0)
    frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    videoFPS = int(cap.get(cv2.CAP_PROP_FPS))

    print (f"frameCount: {frameCount}")
    print (f"frameWidth: {frameWidth}")
    print (f"frameHeight: {frameHeight}")
    print (f"videoFPS: {videoFPS}")

    buf = np.empty((
        frameCount,
        frameHeight,
        frameWidth,
        3), np.dtype('uint8'))

    fc = 0
    ret = True

    while (fc < frameCount):
        ret, buf[fc] = cap.read()
        fc += 1

    cap.release()
    videoArray = buf

    print (f"DURATION: {frameCount/videoFPS}")
    return videoArray

In [3]:
def getMoves(movesPath):
    with open(movesPath, "r") as json_file:
        listOfMoves = []
        for line in json_file.readlines():
            try:
                moves = json.loads(line.replace(":", '":').replace("{", '{"').replace("], ", '], "'))
                for key in moves.keys():
                    if key == "camera":
                        clean = str(moves[key]).replace("[", "").replace("]", "").split(", ")
                        moves[key] = [float(clean[0]), float(clean[1])]
                    else:
                        moves[key] = int(str(moves[key]).replace("[", "").replace("]", ""))
                listOfMoves.append(moves)
            except Exception as e:
                print(e)
    return listOfMoves

In [9]:
challenges = ["MineRLBasaltBuildVillageHouse-v0", "MineRLBasaltCreateVillageAnimalPen-v0", "MineRLBasaltFindCave-v0", "MineRLBasaltMakeWaterfall-v0"]
movesPath = "../assets/datasets/Agent Moves/"

games = {}
moves = []
for challenge in challenges:
    for file in os.listdir(movesPath + challenge + "/"):
        if ".txt" in file:
            stamp = file.split(".")[0]
            videoFile = os.listdir(movesPath + challenge + "/" + stamp + ".mp4/")[0]
            moves += getMoves(movesPath + challenge + "/" + file)
            games.update({stamp: {"moves": getMoves(movesPath + challenge + "/" + file), "frames": getFrames(movesPath + challenge + "/" + stamp + ".mp4/" + videoFile)}})

df = pd.DataFrame.from_dict(moves)

frameCount: 14401
frameWidth: 640
frameHeight: 360
videoFPS: 30
DURATION: 480.03333333333336
frameCount: 5598
frameWidth: 640
frameHeight: 360
videoFPS: 30
DURATION: 186.6
frameCount: 14401
frameWidth: 640
frameHeight: 360
videoFPS: 30


KeyboardInterrupt: 

In [12]:
for game in games:
    print(len(games[game]['frames']), len(games[game]['moves']))

14401 14400
5598 5597


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 943782 entries, 0 to 943781
Data columns (total 22 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   attack     943782 non-null  int64 
 1   back       943782 non-null  int64 
 2   forward    943782 non-null  int64 
 3   jump       943782 non-null  int64 
 4   left       943782 non-null  int64 
 5   right      943782 non-null  int64 
 6   sneak      943782 non-null  int64 
 7   sprint     943782 non-null  int64 
 8   use        943782 non-null  int64 
 9   drop       943782 non-null  int64 
 10  inventory  943782 non-null  int64 
 11  hotbar.1   943782 non-null  int64 
 12  hotbar.2   943782 non-null  int64 
 13  hotbar.3   943782 non-null  int64 
 14  hotbar.4   943782 non-null  int64 
 15  hotbar.5   943782 non-null  int64 
 16  hotbar.6   943782 non-null  int64 
 17  hotbar.7   943782 non-null  int64 
 18  hotbar.8   943782 non-null  int64 
 19  hotbar.9   943782 non-null  int64 
 20  came

In [7]:
df.describe()

,attack,back,forward,jump,left,right,sneak,sprint,use,drop,...,hotbar.1,hotbar.2,hotbar.3,hotbar.4,hotbar.5,hotbar.6,hotbar.7,hotbar.8,hotbar.9,ESC
count,943782.000000,943782.000000,943782.000000,943782.000000,943782.000000,943782.000000,943782.000000,943782.000000,943782.000000,943782.000000,...,943782.000000,943782.000000,943782.000000,943782.000000,943782.000000,943782.000000,943782.000000,943782.000000,943782.000000,943782.0
mean,0.284952,0.057727,0.482672,0.191606,0.071797,0.092966,0.014818,0.179749,0.023328,0.000529,...,0.001020,0.000817,0.000691,0.000917,0.000586,0.000337,0.000159,0.000105,0.000074,0.0
std,0.451392,0.233227,0.499700,0.393565,0.258152,0.290385,0.120824,0.383979,0.150945,0.022988,...,0.031927,0.028570,0.026275,0.030260,0.024199,0.018353,0.012606,0.010241,0.008612,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
75%,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0
